### 【問題1】スクラッチを振り返る

ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

・重み、バイアスの初期化

・エポックのループ

・層ごとのノード数の設定

・層ごとの活性化関数の決定

・線形結合の数式のコーディング

・順伝播、逆伝播の数式のコーディング

・損失関数の計算

・重み、バイアスの更新

### 問題2】スクラッチとTensorFlowの対応を考える

以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。



・重み、バイアスの初期化

　→init = tf.global_variables_initializer()

・エポックのループ
　→for epoch in range(num_epochs):

・層ごとにノード数の設定

　→ノード数などパラメータを設定している

・線形結合のコーディング

　→記載変更が変更となったが、処理内容は同様

・損失関数の計算
　→loss_op = tf.reduce_mean(tf.nn.

・重み、バイアスの更新
　→optimizer = tf.train.AdamOptimizer

In [11]:
!pip install --upgrade tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 32kB/s 
     |████████████████████████████████| 512kB 34.1MB/s 
     |████████████████████████████████| 3.8MB 27.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=c2bedf42d80cafe1dc28b601ae15e6e62d4adb7ba77d8323c94088f6421d137e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installa

### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

In [5]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

from google.colab import drive
drive.mount('/content/drive')

# データセットの読み込み
df = pd.read_csv('drive/My Drive/Sprint13/Iris.csv', dtype = None)
# データフレームから条件抽出
#３種とするので下記処理は削除
#df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)

# ラベルをone-hot化）
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y[y == "Iris-setosa"] = 2
y = y.astype(np.int64)[:, np.newaxis]
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y)

#y[y == "Iris-versicolor"] = 0
#y[y == "Iris-virginica"] = 1
#y[y == "Iris-setosa"] = 2
#y = y.astype(np.int64)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1] # → 4
n_samples = X_train.shape[0]

n_classes = 3  #one-hotの３値分類なので変更

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)  
# 目的関数(2種→3種に改変必要)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果(2種→3種に改変必要)
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0, loss : 5.2272, val_loss : 24.0748, acc : 0.083
Epoch 1, loss : 2.8108, val_loss : 13.3855, acc : 0.167
Epoch 2, loss : 1.8528, val_loss : 6.8164, acc : 0.542
Epoch 3, loss : 1.4473, val_loss : 4.6168, acc : 0.542
Epoch 4, loss : 1.1246, val_loss : 4.1487, acc : 0.583
Epoch 5, loss : 0.9354, val_loss : 2.5178, acc : 0.625
Epoch 6, loss : 0.7090, val_loss : 2.2628, acc : 0.625
Epoch 7, loss : 0.5796, val_loss : 2.4890, acc : 0.792
Epoch 8, loss : 0.5044, val_loss : 1.8024, acc : 0.667
Epoch 9, loss : 0.4098, val_loss : 2.0921, acc : 0.792
Epoch 10, loss : 0.3472, val_loss : 1.3589, acc : 0.750
Epoch 11, loss : 0.2925, val_loss : 1.1582, acc : 0.750
Epoch 12, loss :

### 【問題4】House Pricesのモデルを作成

In [6]:
# データセットの読み込み
#dataset_path ="./house_prices_advanced_regression_techniques/train.csv"
df_house = pd.read_csv('drive/My Drive/Sprint13/train.csv', dtype = None)

# データフレームから条件抽出
y = df_house[['SalePrice']]
X = df_house[['GrLivArea','YearBuilt']]
y = np.array(np.log1p(y))
X = np.array(np.log1p(X))

# trainとtestに分割
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = \
train_test_split(X_train, y_train, test_size=0.2, random_state=0)

def regression_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # tf.addと+は等価である
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    
    return layer_output

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の読み込み                               
logits = regression_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.square(logits - Y))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 指標値計算
mean_square_error = tf.reduce_mean(tf.square(logits - Y))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        #total_loss = 0
        total_mse = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            
            mse = sess.run(mean_square_error,
                                 feed_dict={X: mini_batch_x, Y: mini_batch_y}
            )
            #total_loss += loss
            total_mse += mse
            
        #total_loss /= n_samples
        total_mse /= n_samples
        
        val_mse = sess.run(mean_square_error,
                                     feed_dict={X: X_val, Y: y_val}
        )
        print("Epoch {},  mse : {:.3f}, val_mse : {:.3f}"
              .format(epoch, mse, val_mse))
    test_mse = sess.run(mean_square_error, feed_dict={X: X_test, Y: y_test})
    print("test_mse : {:.3f}".format(test_mse))

Epoch 0,  mse : 13.365, val_mse : 17.323
Epoch 1,  mse : 2.984, val_mse : 3.414
Epoch 2,  mse : 0.450, val_mse : 0.585
Epoch 3,  mse : 0.193, val_mse : 0.361
Epoch 4,  mse : 0.245, val_mse : 0.432
Epoch 5,  mse : 0.354, val_mse : 0.564
Epoch 6,  mse : 0.275, val_mse : 0.466
Epoch 7,  mse : 0.182, val_mse : 0.347
Epoch 8,  mse : 0.304, val_mse : 0.502
Epoch 9,  mse : 0.162, val_mse : 0.321
test_mse : 0.298


### 【問題5】MNISTのモデルを作成¶

In [7]:
#!pip install --upgrade tensorflow==2.4.1

     |████████████████████████████████| 394.3MB 27kB/s 
     |████████████████████████████████| 471kB 37.0MB/s 
     |████████████████████████████████| 10.6MB 37.6MB/s 
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [37]:
# MNISTデータセットのダウンロード
#from keras.datasets import mnist
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [38]:
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

y_test = y_test.astype(np.int64)[:, np.newaxis]
X_train = np.array(X_train)
X_test =np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train = y_train.reshape(-1, 1) == np.arange(10)
y_test=y_test.reshape(-1, 1) == np.arange(10)


(10000, 784)
(10000, 10)


In [39]:
def split_data(data, permutation, val_size_rate=0.2):
    data = data[permutation]
    val_size = int(len(data) * val_size_rate)
    val = data[:val_size]
    train = data[val_size:]
    return train, val

permutation = np.random.permutation(np.arange(len(X_train)))
X_train, X_val = split_data(X_train, permutation)
y_train, y_val = split_data(y_train, permutation)


(48000, 784)
(48000, 10)


In [42]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 20
num_epochs = 15
n_hidden1 = 128
n_hidden2 = 64
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y,1 ), tf.argmax(logits,1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 494.3287, val_loss : 3419.4294, acc : 0.857
Epoch 1, loss : 111.0291, val_loss : 1956.3787, acc : 0.889
Epoch 2, loss : 60.6219, val_loss : 1403.7091, acc : 0.907
Epoch 3, loss : 37.2614, val_loss : 1109.4105, acc : 0.914
Epoch 4, loss : 24.5054, val_loss : 954.5103, acc : 0.920
Epoch 5, loss : 16.8326, val_loss : 881.5616, acc : 0.921
Epoch 6, loss : 11.8580, val_loss : 774.0517, acc : 0.925
Epoch 7, loss : 8.6924, val_loss : 707.9290, acc : 0.927
Epoch 8, loss : 6.5745, val_loss : 665.4524, acc : 0.928
Epoch 9, loss : 4.8952, val_loss : 636.8006, acc : 0.931
Epoch 10, loss : 3.8880, val_loss : 606.1075, acc : 0.934
Epoch 11, loss : 3.0345, val_loss : 598.6090, acc : 0.935
Epoch 12, loss : 2.4870, val_loss : 559.7744, acc : 0.935
Epoch 13, loss : 1.9846, val_loss : 541.8394, acc : 0.937
Epoch 14, loss : 1.6857, val_loss : 535.0844, acc : 0.939
test_acc : 0.942
